In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14233,2022-01-01T17:00:00,ITA,19,Sicilia,38.115697,13.362357,768,102,870,46895,...,DECEDUTI: N. 3 IL 31/12/21 - N. 8 IL 30/12/21 ...,7.0,NaN,NaN,378368.0,0.0,3845222.0,4617827.0,ITG,ITG1
14234,2022-01-01T17:00:00,ITA,9,Toscana,43.769231,11.255889,653,81,734,88142,...,NaN,10.0,NaN,NaN,365561.0,31032.0,5471303.0,3733267.0,ITI,ITI1
14235,2022-01-01T17:00:00,ITA,10,Umbria,43.106758,12.388247,133,8,141,22683,...,NaN,0.0,NaN,NaN,84059.0,8621.0,1354936.0,1568459.0,ITI,ITI2
14236,2022-01-01T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,32,3,35,2444,...,NaN,0.0,NaN,NaN,13626.0,2948.0,126191.0,234546.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14233,2022-01-01T17:00:00,19,Sicilia,768,102,870,46895,47765,5183,5764,...,378368,8463049,3038206.0,7.0,378368.0,0.0,3845222.0,4617827.0,ITG,ITG1
14234,2022-01-01T17:00:00,9,Toscana,653,81,734,88142,88876,14576,14994,...,396593,9204570,3677834.0,10.0,365561.0,31032.0,5471303.0,3733267.0,ITI,ITI1
14235,2022-01-01T17:00:00,10,Umbria,133,8,141,22683,22824,3421,3838,...,92680,2923395,537764.0,0.0,84059.0,8621.0,1354936.0,1568459.0,ITI,ITI2
14236,2022-01-01T17:00:00,2,Valle d'Aosta,32,3,35,2444,2479,350,383,...,16574,360737,111003.0,0.0,13626.0,2948.0,126191.0,234546.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2022-01-01', '2021-12-31')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2022-01-01T17:00:00,1,Piemonte,1242,108,1350,77190,78540,5722,7450,...,502219,11404748,3096619.0,12.0,401949.0,100270.0,4166430.0,7238318.0,ITC,ITC1
1,2022-01-01T17:00:00,2,Valle d'Aosta,32,3,35,2444,2479,350,383,...,16574,360737,111003.0,0.0,13626.0,2948.0,126191.0,234546.0,ITC,ITC2
2,2022-01-01T17:00:00,3,Lombardia,1945,222,2167,282730,284897,32517,37270,...,1254634,25139075,6861289.0,25.0,985733.0,268901.0,13064951.0,12074124.0,ITC,ITC4
3,2022-01-01T17:00:00,5,Veneto,1134,184,1318,102573,103891,10593,14270,...,659993,19733474,2469045.0,20.0,590400.0,69593.0,8321068.0,11412406.0,ITH,ITH3
4,2022-01-01T17:00:00,6,Friuli Venezia Giulia,275,28,303,13545,13848,1328,2331,...,158419,4551164,968465.0,5.0,137009.0,21410.0,2657044.0,1894120.0,ITH,ITH4
5,2022-01-01T17:00:00,7,Liguria,494,48,542,12890,13432,818,2432,...,151492,3567682,1050936.0,6.0,151492.0,0.0,2002034.0,1565648.0,ITC,ITC3
6,2022-01-01T17:00:00,8,Emilia-Romagna,1359,124,1483,87383,88866,11408,12255,...,549175,11720539,2401783.0,16.0,541890.0,7285.0,7316502.0,4404037.0,ITH,ITH5
7,2022-01-01T17:00:00,9,Toscana,653,81,734,88142,88876,14576,14994,...,396593,9204570,3677834.0,10.0,365561.0,31032.0,5471303.0,3733267.0,ITI,ITI1
8,2022-01-01T17:00:00,10,Umbria,133,8,141,22683,22824,3421,3838,...,92680,2923395,537764.0,0.0,84059.0,8621.0,1354936.0,1568459.0,ITI,ITI2
9,2022-01-01T17:00:00,11,Marche,223,47,270,7491,7761,121,1619,...,147083,2026805,1192226.0,5.0,147083.0,0.0,1652434.0,374371.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-12-31T17:00:00,1,Piemonte,1223,107,1330,71488,72818,8932,11501,...,494769,11346311,3081868.0,14.0,400067.0,94702.0,4153392.0,7192919.0,ITC,ITC1
1,2021-12-31T17:00:00,2,Valle d'Aosta,34,3,37,2092,2129,264,350,...,16191,357381,109850.0,1.0,13462.0,2729.0,125719.0,231662.0,ITC,ITC2
2,2021-12-31T17:00:00,3,Lombardia,1859,221,2080,250300,252380,37226,41458,...,1217364,24938879,6824465.0,23.0,971903.0,245461.0,13007625.0,11931254.0,ITC,ITC4
3,2021-12-31T17:00:00,5,Veneto,1173,178,1351,91947,93298,6409,9028,...,645723,19570657,2461424.0,14.0,585689.0,60034.0,8292918.0,11277739.0,ITH,ITH3
4,2021-12-31T17:00:00,6,Friuli Venezia Giulia,287,28,315,12205,12520,1524,2361,...,156092,4524202,964506.0,5.0,135185.0,20907.0,2647643.0,1876559.0,ITH,ITH4
5,2021-12-31T17:00:00,7,Liguria,482,47,529,12085,12614,367,2420,...,149060,3543212,1046131.0,6.0,149060.0,0.0,1993041.0,1550171.0,ITC,ITC3
6,2021-12-31T17:00:00,8,Emilia-Romagna,1346,116,1462,75996,77458,9134,10167,...,536922,11668964,2395965.0,9.0,533348.0,3574.0,7290630.0,4378334.0,ITH,ITH5
7,2021-12-31T17:00:00,9,Toscana,636,76,712,73588,74300,16261,16886,...,381599,9137847,3655266.0,5.0,358538.0,23061.0,5448312.0,3689535.0,ITI,ITI1
8,2021-12-31T17:00:00,10,Umbria,122,9,131,19272,19403,2351,2594,...,88842,2897162,529595.0,2.0,82545.0,6297.0,1348689.0,1548473.0,ITI,ITI2
9,2021-12-31T17:00:00,11,Marche,220,42,262,7378,7640,423,2079,...,145464,2008528,1178055.0,0.0,145464.0,0.0,1644738.0,363790.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2022-01-01T17:00:00,1,Piemonte,1242,108,1350,77190,78540,5722,7450,...,11404748,3096619.0,12.0,401949.0,100270.0,4166430.0,7238318.0,ITC,ITC1,4.341375e+06
1,2022-01-01T17:00:00,2,Valle d'Aosta,32,3,35,2444,2479,350,383,...,360737,111003.0,0.0,13626.0,2948.0,126191.0,234546.0,ITC,ITC2,1.255010e+05
2,2022-01-01T17:00:00,3,Lombardia,1945,222,2167,282730,284897,32517,37270,...,25139075,6861289.0,25.0,985733.0,268901.0,13064951.0,12074124.0,ITC,ITC4,1.010397e+07
3,2022-01-01T17:00:00,5,Veneto,1134,184,1318,102573,103891,10593,14270,...,19733474,2469045.0,20.0,590400.0,69593.0,8321068.0,11412406.0,ITH,ITH3,4.907704e+06
4,2022-01-01T17:00:00,6,Friuli Venezia Giulia,275,28,303,13545,13848,1328,2331,...,4551164,968465.0,5.0,137009.0,21410.0,2657044.0,1894120.0,ITH,ITH4,1.211357e+06
5,2022-01-01T17:00:00,7,Liguria,494,48,542,12890,13432,818,2432,...,3567682,1050936.0,6.0,151492.0,0.0,2002034.0,1565648.0,ITC,ITC3,1.543127e+06
6,2022-01-01T17:00:00,8,Emilia-Romagna,1359,124,1483,87383,88866,11408,12255,...,11720539,2401783.0,16.0,541890.0,7285.0,7316502.0,4404037.0,ITH,ITH5,4.467118e+06
7,2022-01-01T17:00:00,9,Toscana,653,81,734,88142,88876,14576,14994,...,9204570,3677834.0,10.0,365561.0,31032.0,5471303.0,3733267.0,ITI,ITI1,3.722729e+06
8,2022-01-01T17:00:00,10,Umbria,133,8,141,22683,22824,3421,3838,...,2923395,537764.0,0.0,84059.0,8621.0,1354936.0,1568459.0,ITI,ITI2,8.802850e+05
9,2022-01-01T17:00:00,11,Marche,223,47,270,7491,7761,121,1619,...,2026805,1192226.0,5.0,147083.0,0.0,1652434.0,374371.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2022-01-01T17:00:00,1,Piemonte,1242,108,1350,77190,78540,5722,7450,...,7238318.0,ITC,ITC1,4.341375e+06,Piemonte,1242,5824,108,628,99
1,2022-01-01T17:00:00,2,Valle d'Aosta,32,3,35,2444,2479,350,383,...,234546.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,32,99,3,33,0
2,2022-01-01T17:00:00,3,Lombardia,1945,222,2167,282730,284897,32517,37270,...,12074124.0,ITC,ITC4,1.010397e+07,Lombardia,1945,10457,222,1530,0
3,2022-01-01T17:00:00,5,Veneto,1134,184,1318,102573,103891,10593,14270,...,11412406.0,ITH,ITH3,4.907704e+06,Veneto,1134,6000,184,1000,0
4,2022-01-01T17:00:00,6,Friuli Venezia Giulia,275,28,303,13545,13848,1328,2331,...,1894120.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,275,1277,28,175,0
5,2022-01-01T17:00:00,7,Liguria,494,48,542,12890,13432,818,2432,...,1565648.0,ITC,ITC3,1.543127e+06,Liguria,494,1764,48,219,12
6,2022-01-01T17:00:00,8,Emilia-Romagna,1359,124,1483,87383,88866,11408,12255,...,4404037.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,1359,9001,124,889,0
7,2022-01-01T17:00:00,9,Toscana,653,81,734,88142,88876,14576,14994,...,3733267.0,ITI,ITI1,3.722729e+06,Toscana,653,5033,81,570,26
8,2022-01-01T17:00:00,10,Umbria,133,8,141,22683,22824,3421,3838,...,1568459.0,ITI,ITI2,8.802850e+05,Umbria,133,662,8,86,41
9,2022-01-01T17:00:00,11,Marche,223,47,270,7491,7761,121,1619,...,374371.0,ITI,ITI3,1.518400e+06,Marche,223,995,47,252,51


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2022-01-01T17:00:00,3,Lombardia,1945,222,2167,282730,284897,32517,37270,...,14.51,32430,-4188,200196,36824.0,14,4739,18.62,0.36886,12.42
1,2022-01-01T17:00:00,9,Toscana,653,81,734,88142,88876,14576,14994,...,14.21,14554,-1892,66723,22568.0,4,414,22.47,0.40277,10.65
2,2022-01-01T17:00:00,5,Veneto,1134,184,1318,102573,103891,10593,14270,...,18.40,10626,5242,162817,7621.0,12,3665,8.76,0.29077,13.45
3,2022-01-01T17:00:00,15,Campania,686,55,741,89648,90389,12860,13888,...,9.43,12842,-699,132821,1791.0,4,1024,10.46,0.24003,10.32
4,2022-01-01T17:00:00,12,Lazio,1154,153,1307,82174,83481,11360,12345,...,16.22,11369,3868,113438,6312.0,6,979,10.88,0.21047,8.82
5,2022-01-01T17:00:00,8,Emilia-Romagna,1359,124,1483,87383,88866,11408,12255,...,13.95,11387,2088,51575,5818.0,15,830,23.76,0.27434,12.29
6,2022-01-01T17:00:00,1,Piemonte,1242,108,1350,77190,78540,5722,7450,...,17.20,5702,-4051,58437,14751.0,9,1719,12.75,0.17160,11.57
7,2022-01-01T17:00:00,19,Sicilia,768,102,870,46895,47765,5183,5764,...,11.94,5160,301,56390,20826.0,12,569,10.22,0.11601,7.62
8,2022-01-01T17:00:00,13,Abruzzo,179,21,200,20673,20873,3822,4022,...,11.60,3820,-751,53778,8810.0,0,200,7.48,0.30802,8.47
9,2022-01-01T17:00:00,10,Umbria,133,8,141,22683,22824,3421,3838,...,9.30,3411,1244,26233,8169.0,0,417,14.63,0.43600,10.53


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,P.A. Trento,20,70,90,22.22
1,Liguria,48,171,219,21.92
2,Marche,47,205,252,18.65
3,Veneto,184,816,1000,18.40
4,Piemonte,108,520,628,17.20
5,P.A. Bolzano,17,83,100,17.00
6,Lazio,153,790,943,16.22
7,Friuli Venezia Giulia,28,147,175,16.00
8,Calabria,28,161,189,14.81
9,Lombardia,222,1308,1530,14.51


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))